在第十三天的學習計劃中，我們將深入了解如何通過 **微調預訓練模型** 來提高模型的準確性。具體來說，我們將探索如何使用 **凍結與解凍層技術**，這是遷移學習中常見的一種策略，通過凍結部分層來保持預訓練模型中學到的知識，然後逐步解凍一些層進行微調，以適應新的任務需求。

---

### **1. 什麼是微調模型？**

**微調（Fine-tuning）** 是指在預訓練模型的基礎上，進行進一步的訓練，以適應新的數據或任務。在大部分情況下，預訓練模型已經在大型數據集（如 ImageNet）上學到了豐富的特徵表示，這些特徵可以用於其他類似的任務中。微調可以讓模型更好地適應特定的數據分佈，進一步提升性能。

#### **微調模型的步驟：**
1. **加載預訓練模型**：使用一個在大數據集上預訓練好的模型（如 ResNet、VGG）。
2. **凍結部分層**：保留預訓練模型的前幾層，這些層通常學到了較為通用的特徵（如邊緣、形狀）。
3. **修改輸出層**：將原來的分類層替換為適應新任務的分類器（如改為 10 類）。
4. **解凍部分層**：在訓練過程中逐步解凍一些層，讓模型進行微調，從而學習到與新任務相關的高級特徵。

---

### **2. 凍結與解凍層技術**

#### **2.1 凍結層**

在微調模型時，常見的做法是 **凍結** 預訓練模型的前幾層，使這些層的權重保持不變，不參與訓練。這樣可以保留預訓練模型中學到的通用特徵，同時減少計算成本。

#### **2.2 解凍層**

當你開始進行微調時，可以逐步 **解凍** 一些卷積層，使它們參與訓練。解凍的層數取決於任務的具體需求。解凍更多層可以讓模型適應新任務的特定特徵，但同時也可能增加過擬合的風險，特別是當數據量較少時。

---

### **3. 實作微調預訓練模型**

以下是一個完整的 Keras 實作範例，我們將使用 **ResNet50** 進行微調，並展示如何凍結和解凍模型的層來提高性能。這個範例基於 CIFAR-10 數據集，你可以根據需要替換為其他數據集。

#### **步驟**：
1. 加載 ResNet50 預訓練模型（不包含頂層）。
2. 凍結 ResNet50 的大部分層，並添加新的分類器層。
3. 進行初步訓練。
4. 解凍部分 ResNet50 層，進行微調。


#### **Keras 實作：微調 ResNet50**

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.callbacks import ModelCheckpoint

# 下載並預處理 CIFAR-10 資料集
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# 將標籤轉為 one-hot 編碼
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# 加載 ResNet50 預訓練模型，不包括頂層（分類層）
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# 凍結 ResNet50 的所有層
for layer in base_model.layers:
    layer.trainable = False

# 添加新的分類器層
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')  # CIFAR-10 共有 10 個類別
])

# 編譯模型
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 初步訓練分類層
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))

# 解凍 ResNet50 的後 10 層，進行微調
for layer in base_model.layers[-10:]:
    layer.trainable = True

# 使用較小的學習率進行微調
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), 
              loss='categorical_crossentropy', metrics=['accuracy'])

# 訓練微調後的模型
model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test))


### **4. 進階策略**

#### **4.1 逐步解凍**

在進行微調時，**逐步解凍**是常見的策略之一。這種策略可以讓你在每個階段解凍幾層，從而逐步讓模型學習新任務的特徵。這樣可以減少模型突然大幅更新權重，導致不穩定的訓練過程。

#### **4.2 使用小學習率**

在微調過程中，使用 **較小的學習率** 是至關重要的。這是因為預訓練模型中的權重已經在大型數據集上經過了長時間的優化，只需要對這些權重進行輕微調整來適應新任務。如果學習率太大，可能會破壞原本已經學到的特徵。

#### **4.3 使用權重衰減**

在微調時，**權重衰減** 也是一個有用的技術。這可以防止模型在微調過程中過度適應訓練數據，從而幫助模型提高在測試集上的泛化能力。

---

### **5. 使用 PyTorch 實作微調預訓練模型**

我們也可以使用 PyTorch 來實現相同的微調流程。以下是一個 PyTorch 實作範例，展示如何使用 ResNet18 進行微調。

#### **PyTorch 實作：微調 ResNet18**

---

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

# 資料預處理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# CIFAR-10 資料集
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 加載 ResNet18 預訓練模型
model = models.resnet18(pretrained=True)

# 凍結所有層
for param in model.parameters():
    param.requires_grad = False

# 替換最後的全連接層
num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_features, 128),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 10)
)

# 使用 Adam 優化器
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# 訓練模型
for epoch in range(5):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')

# 解凍 ResNet18 的後幾層
for param in model.layer4.parameters():
    param.requires_grad = True

# 使用較小學習率進行微調
optimizer = optim.Adam(model.parameters(), lr=1e-5)

# 微調模型
for epoch in range(10):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')


### **6. 總結**

1. **微調模型** 是提高預訓練模型性能的重要技術。通過凍結和解凍層的策略，可以有效保留預訓練模型的知識，並適應新的數據。
2. **凍結層** 保留預訓練模型中學到的通用特徵，而 **解凍部分層** 允許模型根據新的任務進行微調。
3. **逐步解凍和使用較小學習率** 是微調過程中的關鍵技術，它們有助於提高模型在新數據集上的性能。

---